In [1]:
!pip install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 5.7 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [1]:
import json
from transformers import T5Tokenizer, T5ForConditionalGeneration
import torch
import accelerate

In [2]:
tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-large")
model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-large", device_map="auto")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have

In [3]:
def evaluate(answer: str, predicted: str):
    return answer.lower() in predicted.lower()

In [4]:
with open('/content/updated_concept_net.dev.csqa.json', 'r') as file:
    data = json.load(file)

In [5]:
from tqdm import tqdm

results = []
correct_count = 0  # Initialize correct answer count
incorrect_count = 0  # Initialize incorrect answer count
total_count = 0

for record in tqdm(data):
    try:
        # Generate the input text for the model
        input_text = f"Answer the question: {record['query']}"
        input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to("cuda")

        # Generate the model's output and decode it
        outputs = model.generate(input_ids)
        predicted_answer = tokenizer.decode(outputs[0], skip_special_tokens=True)

        # Evaluate the prediction
        correct = evaluate(record['answer'], predicted_answer)
        total_count += 1
        if correct:
            correct_count += 1
        else:
            incorrect_count += 1

        # Append results with new fields "ok" and "predicted"
        record['ok'] = 1 if correct else 0
        record['predicted'] = predicted_answer
        results.append(record)

        # Calculate and print the current accuracy
        current_accuracy = correct_count / total_count
        print(f"Current Accuracy after processing {total_count} records: {current_accuracy:.2f}")

    except Exception as e:
        print(f"Error processing record {record['query']}: {str(e)}")
        continue

# Final statistics
print(f"Total Records Processed: {total_count}")
print(f"Number of Correct Answers: {correct_count}")
print(f"Number of Wrong Answers: {incorrect_count}")
accuracy = correct_count / total_count
print(f"Final Accuracy: {accuracy:.2f}")

# Optionally, write the results to a new JSON file
with open('output_with_results.json', 'w') as outfile:
    json.dump(results, outfile, indent=4)
print("Processing complete. Results saved to 'output_with_results.json'.")

  0%|          | 0/1221 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1141: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
  0%|          | 1/1221 [00:06<2:13:40,  6.57s/it]

Current Accuracy after processing 1 records: 0.00


  0%|          | 3/1221 [00:07<33:10,  1.63s/it]

Current Accuracy after processing 2 records: 0.00
Current Accuracy after processing 3 records: 0.00


  0%|          | 4/1221 [00:07<22:40,  1.12s/it]

...

Current Accuracy after processing 1218 records: 0.10


100%|█████████▉| 1220/1221 [04:30<00:00,  5.18it/s]

Current Accuracy after processing 1219 records: 0.10
Current Accuracy after processing 1220 records: 0.10


100%|██████████| 1221/1221 [04:30<00:00,  4.51it/s]

Current Accuracy after processing 1221 records: 0.10
Total Records Processed: 1221
Number of Correct Answers: 117
Number of Wrong Answers: 1104
Final Accuracy: 0.10
Processing complete. Results saved to 'output_with_results.json'.
